In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('/Users/chentingkao/PracticeData/Hitters.csv')
df.head()

,Unnamed: 0,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,...,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,-Andy Allanson,293,66,1,30,29,14,1,293,66,...,30,29,14,A,E,446,33,20,NaN,A
1,-Alan Ashby,315,81,7,24,38,39,14,3449,835,...,321,414,375,N,W,632,43,10,475.0,N
2,-Alvin Davis,479,130,18,66,72,76,3,1624,457,...,224,266,263,A,W,880,82,14,480.0,A
3,-Andre Dawson,496,141,20,65,78,37,11,5628,1575,...,828,838,354,N,E,200,11,3,500.0,N
4,-Andres Galarraga,321,87,10,39,42,30,2,396,101,...,48,46,33,N,E,805,40,4,91.5,N


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 21 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  322 non-null    object 
 1   AtBat       322 non-null    int64  
 2   Hits        322 non-null    int64  
 3   HmRun       322 non-null    int64  
 4   Runs        322 non-null    int64  
 5   RBI         322 non-null    int64  
 6   Walks       322 non-null    int64  
 7   Years       322 non-null    int64  
 8   CAtBat      322 non-null    int64  
 9   CHits       322 non-null    int64  
 10  CHmRun      322 non-null    int64  
 11  CRuns       322 non-null    int64  
 12  CRBI        322 non-null    int64  
 13  CWalks      322 non-null    int64  
 14  League      322 non-null    object 
 15  Division    322 non-null    object 
 16  PutOuts     322 non-null    int64  
 17  Assists     322 non-null    int64  
 18  Errors      322 non-null    int64  
 19  Salary      263 non-null    f

In [22]:
# Fill NA with median values
df['Salary'].fillna(df['Salary'].median(skipna = True), inplace = True)
df.isna().sum()

Unnamed: 0    0
AtBat         0
Hits          0
HmRun         0
Runs          0
RBI           0
Walks         0
Years         0
CAtBat        0
CHits         0
CHmRun        0
CRuns         0
CRBI          0
CWalks        0
League        0
Division      0
PutOuts       0
Assists       0
Errors        0
Salary        0
NewLeague     0
dtype: int64

In [14]:
# turn categorical values into numeric values with one-hot encoding
one_hot_transform_features = pd.get_dummies(df[['League', 'NewLeague', 'Division']], drop_first = True)

In [18]:
target = df['Salary']
part_feature_df = df.drop(['Unnamed: 0', 'League', 'NewLeague', 'Division', 'Salary'], axis = 1).astype('float64')

# use pd.concat to combine two dataframes
full_feature_df = pd.concat([part_feature_df, one_hot_transform_features[['NewLeague_N', 'Division_W', 'League_N']]], axis = 1)

In [25]:
# run Lasso model and measure r2
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score

X_train, X_test, y_train, y_test = train_test_split(full_feature_df, target, test_size = 0.25, random_state = 42)

# where is lambda?
lasso_model = Lasso()
lasso_model.fit(X_train, y_train)
print(lasso_model.intercept_)
print(lasso_model.coef_)

y_pred = lasso_model.predict(X_test)
print(r2_score(y_test, y_pred))

342.87359059300354
[-1.98559338e+00  5.50496531e+00  4.79614112e+00  1.02110790e-01
 -8.11524918e-01  4.87004824e+00 -9.97815019e+00 -2.19389480e-01
  6.16229898e-01  9.02275210e-03  8.73994376e-01  7.84176055e-01
 -8.13425077e-01  1.83989529e-01  4.04846125e-01 -4.08650304e+00
 -0.00000000e+00 -1.11463312e+02  2.67091100e+01]
0.3657514287242595


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.923e+06, tolerance: 3.853e+03
  model = cd_fast.enet_coordinate_descent(


In [31]:
from sklearn.linear_model import LassoCV

# cv=10 means 10-fold cross-validation, which divides the data into 10 parts,
# trains the model on 9 parts, and tests it on the remaining part. This process
# is repeated 10 times, each time with a different part as the test set.

# max_iter=10000: This sets the maximum number of iterations for the optimization
# algorithm to converge

# n_jobs=-1: This parameter allows for parallel computation. Setting n_jobs=-1
# instructs the algorithm to use all available CPU cores, which can
# significantly speed up the cross-validation process, especially with large
# datasets and complex models.
lasso_cv = LassoCV(alphas = np.random.randint(0, 1000, 100), cv = 10, max_iter = 10000, n_jobs = -1).fit(X_train, y_train)
print(lasso_cv.alpha_)

lasso_tuned = Lasso().set_params(alpha = lasso_cv.alpha_).fit(X_train, y_train)

# the correlation coefficient of some features becomes zero
pd.Series(lasso_tuned.coef_, index = X_train.columns)

8


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.910e+06, tolerance: 3.853e+03
  model = cd_fast.enet_coordinate_descent(


AtBat          -1.958434
Hits            5.280894
HmRun           3.279343
Runs            0.194691
RBI            -0.267804
Walks           4.886604
Years          -7.026429
CAtBat         -0.239349
CHits           0.687092
CHmRun          0.105348
CRuns           0.847058
CRBI            0.738894
CWalks         -0.804742
PutOuts         0.181957
Assists         0.390156
Errors         -3.158250
NewLeague_N     0.000000
Division_W    -82.227023
League_N        0.000000
dtype: float64